# Import Required Packages

In [ ]:
from Jupyter import *
import os
from time import sleep
from virus_total_apis import PublicApi as VirusTotalPublicApi
import json
from getpass import getpass

# Check If folders Exist

In [ ]:
MalAnalyst.checkFolder("Samples")
MalAnalyst.checkFolder("Defanged")
empty = MalAnalyst.checkSamples("Samples")
if empty:
    print("No Samples Provided, Need Samples")

# Download Samples From Malware Bazaar?

# Enumerate Samples

In [ ]:
samples=!ls Samples/*
for s in samples:
    print("Samples = " + s)

In [ ]:
sampleObj = [MalAnalyst(s) for s in samples]

# Create a subdirectory for the Malware

In [ ]:
for obj in sampleObj:
    createSub = MalAnalyst.createSubDirectoryForMalwareSample(obj.sampleName)
    hash = MalAnalyst.retrieveSha256HashSum(obj.sampleName)
    print(createSub)
    print("File hash is: " + hash)
    obj.hash = hash
    obj.hashedPath = createSub

# Check Details on Virus-Total

In [ ]:
VT_API_KEY = getpass("Enter VirusTotal API Key (leave blank if none)")

In [ ]:
if VT_API_KEY:
    virustot = VirusTotalPublicApi(VT_API_KEY)
else:
    print("No Virus Total Key Inputted...Skipping")

In [ ]:
if VT_API_KEY:
    for obj in sampleObj:
        print("Virus Total info regarding this hashsum: " + obj.hash)
        res = virustot.get_file_report(obj.hash)
        conf = Utils.maliciousConfidence(res)
        print("Confidence = " + str(conf))
        
        if len(sampleObj) >= 5:
            sleep(16)
        
    
else:
    print("No API KEY Skipping...")

# Defang Malicious Binary

In [ ]:
for obj in sampleObj:
    defangBinary = MalAnalyst.moveAndDefang(obj.sampleName)
    print(defangBinary)
    obj.newSampleName = defangBinary

# Create a automatic yara rule

In [ ]:
%%capture
for obj in sampleObj:
    MalAnalyst.retrieveStringsFromYarGen(obj.hashedPath, obj.hash)

In [ ]:
print("YaraGen Rules creation has been completed")

# Pull Strings from Binary

In [ ]:
for obj in sampleObj:
    MalAnalyst.pullStringsEncodedAndUnicode(obj.newSampleName, obj.hashedPath)  #obj.samplename 
    print("Encoded Strings and Unicode files for " + obj.hash + "has been pulled")

# Zip Malware with a password

In [ ]:
for obj in sampleObj:
    MalAnalyst.zipMaliciousSample(obj.newSampleName, obj.hash, obj.hashedPath)
    print(obj.hash + " has been zipped and protected with the password infected")    